**Set Environment**

In [1]:
suppressMessages(suppressWarnings(source("config.R")))

You are on Duke Server: DCC
BASE DIRECTORY:     /work/kk319 
WORK DIRECTORY:     /work/kk319/out/CombEffect_STARR 
LOG  DIRECTORY:     /work/kk319/out/CombEffect_STARR/log 
PATH OF SOURCE:     /work/kk319/source 
PATH OF EXECUTABLE: /work/kk319/exe 
PATH OF ANNOTATION: /work/kk319/annotation 


In [2]:
fdiry = file.path(FD_WRK, "annotation_fragment")
motif = "NR_20"
fname = "target_PER1.bed"
fglob = file.path(fdiry, "*", motif, fname)
Sys.glob(fglob)

[1] "/work/kk319/out/CombEffect_STARR/annotation_fragment/Input/NR_20/target_PER1.bed"    
 [2] "/work/kk319/out/CombEffect_STARR/annotation_fragment/Input1/NR_20/target_PER1.bed"   
 [3] "/work/kk319/out/CombEffect_STARR/annotation_fragment/Input2/NR_20/target_PER1.bed"   
 [4] "/work/kk319/out/CombEffect_STARR/annotation_fragment/Input3/NR_20/target_PER1.bed"   
 [5] "/work/kk319/out/CombEffect_STARR/annotation_fragment/Input4/NR_20/target_PER1.bed"   
 [6] "/work/kk319/out/CombEffect_STARR/annotation_fragment/Input5/NR_20/target_PER1.bed"   
 [7] "/work/kk319/out/CombEffect_STARR/annotation_fragment/TFX2_DMSO/NR_20/target_PER1.bed"
 [8] "/work/kk319/out/CombEffect_STARR/annotation_fragment/TFX2_Dex/NR_20/target_PER1.bed" 
 [9] "/work/kk319/out/CombEffect_STARR/annotation_fragment/TFX3_DMSO/NR_20/target_PER1.bed"
[10] "/work/kk319/out/CombEffect_STARR/annotation_fragment/TFX3_Dex/NR_20/target_PER1.bed" 
[11] "/work/kk319/out/CombEffect_STARR/annotation_fragment/TFX4_DMSO/NR_20/target_PER1.bed"
[12] "/work/kk319/out/CombEffect_STARR/annotation_fragment/TFX4_Dex/NR_20/target_PER1.bed" 
[13] "/work/kk319/out/CombEffect_STARR/annotation_fragment/TFX5_DMSO/NR_20/target_PER1.bed"
[14] "/work/kk319/out/CombEffect_STARR/annotation_fragment/TFX5_Dex/NR_20/target_PER1.bed" 
[15] "/work/kk319/out/CombEffect_STARR/annotation_fragment/TFX_DMSO/NR_20/target_PER1.bed" 
[16] "/work/kk319/out/CombEffect_STARR/annotation_fragment/TFX_Dex/NR_20/target_PER1.bed"

## Import

In [3]:
get_sample = function(idn_sample){
    idn = idn_sample
    idn = str_replace(
        string = idn, 
        pattern = "Input[0-9]", 
        replacement = "Input")
    idn = str_replace(
        string = idn, 
        pattern = "TFX[0-9]_", 
        replacement="TFX_")
    return(idn)
}

In [4]:
fdiry = file.path(FD_WRK, "source")
fname = "library_size.txt"
fpath = file.path(fdiry, fname)

ctypes = c(col_integer(), col_character())
cnames = c("Size", "Fpath")
dat_lib = read_tsv(fpath, col_types=ctypes, col_names = cnames)
dat_lib = dat_lib %>% 
    mutate(Sample = tools::file_path_sans_ext(basename(Fpath))) %>%
    mutate(Group = get_sample(Sample))
dat_lib = dat_lib %>% dplyr::select(Size, Sample, Group)
head(dat_lib, 10)

Size,Sample,Group
18666630,Input1,Input
20167924,Input2,Input
23280988,Input3,Input
19003938,Input4,Input
15325016,Input5,Input
48376253,TFX2_AZD2906,TFX_AZD2906
52542517,TFX2_AZD9567,TFX_AZD9567
43646484,TFX2_CORT108297,TFX_CORT108297
41732268,TFX2_CpdA,TFX_CpdA
43844606,TFX2_DMSO,TFX_DMSO


## Import

In [5]:
Samples=c(
    paste0("Input", 1:5),
    paste0("TFX",   2:5, "_DMSO"),
    paste0("TFX",   2:5, "_Dex")
)

In [6]:
fpaths = list.dirs("/work/kk319/out/CombEffect_STARR/annotation_fragment/Input1")
motifs = basename(fpaths)
motifs = grep(pattern = paste(c(Samples, "ipynb"), collapse="|"),
     x       = motifs, 
     invert  = TRUE,
     value   = TRUE)

In [42]:
mtf1 = "NR_20"
mtf2 = "AP1_1"
motifs = c(mtf1, mtf2)

In [52]:
ctypes = c(col_character(), col_integer(), col_integer(), col_integer(),
           col_character(), col_integer(), col_integer(),
           col_character(), col_double(),  col_integer())
cnames = c("Chrom_Frag", "Start_Frag", "End_Frag", "Count_Frag",
           "Chrom_MTF",  "Start_MTF",  "End_MTF",
           "Motif", "Score", "Overlap")
fname = "target_PER1.bed"

lst_dat = lapply(motifs, function(motif){
    lst = lapply(Samples, function(sam){
        ### get file
        fdiry = file.path(FD_WRK, "annotation_fragment", sam, motif)
        fpath = file.path(fdiry, fname)
        print(fpath)
        
        ### import data
        dat = read_tsv(fpath, col_types=ctypes, col_names=cnames)
        dat$Sample = sam
        return(dat)
    })
    print("++++++++++++++++++++++++++++++++++++++++++++++++++")
    dat = bind_rows(lst)
    return(dat)
})
names(lst_dat) = motifs


fun = function(dat){nrow(dat) != 0}
lst_dat = Filter(fun, lst_dat)

lst_dat = lapply(lst_dat, function(dat){
    dat$Motif = str_replace_all(
        string  = dat$Motif, 
        pattern = "/", 
        replacement = "_")
    return(dat)
})

[1] "/work/kk319/out/CombEffect_STARR/annotation_fragment/Input1/NR_20/target_PER1.bed"
[1] "/work/kk319/out/CombEffect_STARR/annotation_fragment/Input2/NR_20/target_PER1.bed"
[1] "/work/kk319/out/CombEffect_STARR/annotation_fragment/Input3/NR_20/target_PER1.bed"
[1] "/work/kk319/out/CombEffect_STARR/annotation_fragment/Input4/NR_20/target_PER1.bed"
[1] "/work/kk319/out/CombEffect_STARR/annotation_fragment/Input5/NR_20/target_PER1.bed"
[1] "/work/kk319/out/CombEffect_STARR/annotation_fragment/TFX2_DMSO/NR_20/target_PER1.bed"
[1] "/work/kk319/out/CombEffect_STARR/annotation_fragment/TFX3_DMSO/NR_20/target_PER1.bed"
[1] "/work/kk319/out/CombEffect_STARR/annotation_fragment/TFX4_DMSO/NR_20/target_PER1.bed"
[1] "/work/kk319/out/CombEffect_STARR/annotation_fragment/TFX5_DMSO/NR_20/target_PER1.bed"
[1] "/work/kk319/out/CombEffect_STARR/annotation_fragment/TFX2_Dex/NR_20/target_PER1.bed"
[1] "/work/kk319/out/CombEffect_STARR/annotation_fragment/TFX3_Dex/NR_20/target_PER1.bed"
[1] "/work/kk319

In [54]:
lst_frag = lapply(lst_dat, function(dat){
    tmp = dat %>% 
        dplyr::select(Chrom_Frag, Start_Frag, End_Frag, Count_Frag, Motif, Sample) %>%
        distinct()
    return(tmp)
})

In [59]:
dat1 = bind_rows(lst_frag) %>% 
    dplyr::select(Chrom_Frag, Start_Frag, End_Frag, Count_Frag, Sample) %>%
    distinct

dat2 = lst_frag[["NR_20"]] %>% 
    dplyr::filter(Motif == "NR_20") %>% 
    dplyr::select(Chrom_Frag, Start_Frag, End_Frag, Count_Frag, Sample, Motif)

dat3 = lst_frag[["AP1_1"]] %>% 
    dplyr::filter(Motif == "AP1_1") %>% 
    dplyr::select(Chrom_Frag, Start_Frag, End_Frag, Count_Frag, Sample, Motif)

In [60]:
head(dat1, 3)

Chrom_Frag,Start_Frag,End_Frag,Count_Frag,Sample
chr17,8149351,8150361,1,Input1
chr17,8149418,8150427,1,Input1
chr17,8149468,8150589,1,Input1


In [61]:
head(dat2, 3)

Chrom_Frag,Start_Frag,End_Frag,Count_Frag,Sample,Motif
chr17,8149351,8150361,1,Input1,NR_20
chr17,8149418,8150427,1,Input1,NR_20
chr17,8149468,8150589,1,Input1,NR_20


In [62]:
head(dat3, 3)

Chrom_Frag,Start_Frag,End_Frag,Count_Frag,Sample,Motif
chr17,8148117,8149012,1,Input1,AP1_1
chr17,8148122,8149107,1,Input1,AP1_1
chr17,8148178,8149194,1,Input1,AP1_1


In [63]:
dat = dat1 %>%
        full_join(dat2, by = c("Chrom_Frag", "Start_Frag", "End_Frag", "Count_Frag", "Sample")) %>%
        full_join(dat3, by = c("Chrom_Frag", "Start_Frag", "End_Frag", "Count_Frag", "Sample")) %>%
        mutate(Motif = paste(Motif.x, Motif.y, sep = "_")) %>%
        mutate(Motif = str_remove(string=Motif, pattern="_NA|NA_")) %>% 
        mutate(Group = str_remove(string = Sample, pattern = "[0-9]")) %>%
        mutate(X     = paste(Group, Motif, sep="_")) %>%
        mutate(X     = ifelse(str_detect(X, "Input"), "Input", X))

In [66]:
head(dat,3)

Chrom_Frag,Start_Frag,End_Frag,Count_Frag,Sample,Motif.x,Motif.y,Motif,Group,X
chr17,8149351,8150361,1,Input1,NR_20,AP1_1,NR_20_AP1_1,Input,Input
chr17,8149418,8150427,1,Input1,NR_20,AP1_1,NR_20_AP1_1,Input,Input
chr17,8149468,8150589,1,Input1,NR_20,AP1_1,NR_20_AP1_1,Input,Input


In [67]:
tail(dat,3)

Chrom_Frag,Start_Frag,End_Frag,Count_Frag,Sample,Motif.x,Motif.y,Motif,Group,X
chr17,8158178,8159105,1,TFX5_Dex,NA,AP1_1,AP1_1,TFX_Dex,TFX_Dex_AP1_1
chr17,8158179,8159105,1,TFX5_Dex,NA,AP1_1,AP1_1,TFX_Dex,TFX_Dex_AP1_1
chr17,8158224,8159128,1,TFX5_Dex,NA,AP1_1,AP1_1,TFX_Dex,TFX_Dex_AP1_1


In [68]:
tmp = dat %>% 
    group_by(Sample, X) %>% 
    summarise(Value = sum(Count_Frag), .groups = 'drop')
tmp = tmp %>% left_join(dat_lib, by="Sample")
tmp = tmp %>%
    mutate(Norm_Value    = Value / Size) %>% 
    mutate(Lognorm_Value = log10(Value) - log10(Size))

idx11 = paste("TFX_DMSO", mtf1,       sep="_")
idx12 = paste("TFX_DMSO", mtf2,       sep="_")
idx13 = paste("TFX_DMSO", mtf1, mtf2, sep="_")
idx21 = paste("TFX_Dex",  mtf1,       sep="_")
idx22 = paste("TFX_Dex",  mtf2,       sep="_")
idx23 = paste("TFX_Dex",  mtf1, mtf2, sep="_")
idxs  = c("Input", idx11, idx12, idx21, idx22, idx13, idx23)
tmp$X = factor(tmp$X, levels=idxs)

In [69]:
print(dim(tmp))
head(tmp, 3)

[1] 29  7


Sample,X,Value,Size,Group,Norm_Value,Lognorm_Value
Input1,Input,138,18666630,Input,7.392872e-06,-5.131187
Input2,Input,144,20167924,Input,7.140051e-06,-5.146299
Input3,Input,200,23280988,Input,8.590701e-06,-5.065971


In [70]:
X = model.matrix(~X, tmp)
print(dim(X))

[1] 29  7


In [71]:
idx11 = paste("XTFX_DMSO", mtf1,       sep="_")
idx12 = paste("XTFX_DMSO", mtf2,       sep="_")
idx13 = paste("XTFX_DMSO", mtf1, mtf2, sep="_")
idx21 = paste("XTFX_Dex",  mtf1,       sep="_")
idx22 = paste("XTFX_Dex",  mtf2,       sep="_")
idx23 = paste("XTFX_Dex",  mtf1, mtf2, sep="_")

X = model.matrix(~X, tmp)
X[,idx11] = X[,idx11] + X[,idx13] + X[,idx21] + X[,idx23]
X[,idx12] = X[,idx12] + X[,idx13] + X[,idx22] + X[,idx23]
X[,idx21] = X[,idx21] + X[,idx23]
X[,idx22] = X[,idx22] + X[,idx23]
X[,idx13] = X[,idx13] + X[,idx23]

In [76]:
cbind(X, as.character(tmp$X)) %>% as.data.frame %>% distinct

,(Intercept),XTFX_DMSO_NR_20,XTFX_DMSO_AP1_1,XTFX_Dex_NR_20,XTFX_Dex_AP1_1,XTFX_DMSO_NR_20_AP1_1,XTFX_Dex_NR_20_AP1_1,V8
1,1,0,0,0,0,0,0,Input
6,1,0,1,0,1,0,0,TFX_Dex_AP1_1
7,1,1,0,1,0,0,0,TFX_Dex_NR_20
8,1,1,1,1,1,1,1,TFX_Dex_NR_20_AP1_1
9,1,0,1,0,0,0,0,TFX_DMSO_AP1_1
10,1,1,0,0,0,0,0,TFX_DMSO_NR_20
11,1,1,1,0,0,1,0,TFX_DMSO_NR_20_AP1_1


In [79]:
y = tmp$Norm_Value
fit = lm(y ~ X + 0)
res = summary(fit)
print(res)


Call:
lm(formula = y ~ X + 0)

Residuals:
       Min         1Q     Median         3Q        Max 
-3.019e-06 -7.286e-07 -6.300e-08  4.118e-07  5.841e-06 

Coefficients:
                         Estimate Std. Error t value Pr(>|t|)    
X(Intercept)            7.290e-06  8.841e-07   8.246 3.56e-08 ***
XXTFX_DMSO_NR_20       -9.442e-08  1.326e-06  -0.071   0.9439    
XXTFX_DMSO_AP1_1       -2.871e-06  1.326e-06  -2.165   0.0415 *  
XXTFX_Dex_NR_20         2.381e-05  1.398e-06  17.032 3.71e-14 ***
XXTFX_Dex_AP1_1        -1.073e-06  1.398e-06  -0.767   0.4510    
XXTFX_DMSO_NR_20_AP1_1  5.069e-07  1.927e-06   0.263   0.7949    
XXTFX_Dex_NR_20_AP1_1  -6.264e-06  2.421e-06  -2.587   0.0168 *  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 1.977e-06 on 22 degrees of freedom
Multiple R-squared:  0.9866,	Adjusted R-squared:  0.9824 
F-statistic: 232.1 on 7 and 22 DF,  p-value: < 2.2e-16

